In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
csv_path = "Dataset - Sheet2.csv"
df = pd.read_csv(csv_path)
print(df.columns)
df.head()

Index(['Prompt', 'Response'], dtype='object')


,Prompt,Response
0,I have never done programming in my life. Can ...,"""Yes, this is the perfect bootcamp for anyone ..."
1,Why should I trust Codebasics?,Till now 9000 + learners have benefitted from ...
2,Is there any prerequisite for taking this boot...,"""Our bootcamp is specifically designed for beg..."
3,What datasets are used in this bootcamp? Is it...,The datasets used in this bootcamp are crafted...
4,I’m not sure if this bootcamp is good enough f...,We got you covered. Go ahead and watch our you...


In [3]:
st_model = SentenceTransformer("all-MiniLM-L6-v2")

In [4]:
faq_texts = (
    "Question: " + df["Prompt"].astype(str) +
    "\nAnswer: " + df["Response"].astype(str)
).tolist()

In [5]:
faq_embeddings = st_model.encode(
    faq_texts,
    show_progress_bar=True,
    normalize_embeddings=True
)
faq_embeddings.shape

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

(1100, 384)

In [6]:
def retrieve_similar_faqs(query: str, top_k: int = 4):
    """Return top_k most similar FAQs to the query."""
    query_emb = st_model.encode([query], normalize_embeddings=True)
    sims = cosine_similarity(query_emb, faq_embeddings)[0] 
    top_indices = np.argsort(sims)[::-1][:top_k]
    top_scores = sims[top_indices]
    results = []
    for idx, score in zip(top_indices, top_scores):
        row = df.iloc[idx]
        results.append({
            "index": int(idx),
            "score": float(score),
            "prompt": row["Prompt"],
            "response": row["Response"],
        })
    return results

In [7]:
test_q = "Do you provide job assistance and job guarantee?"
for r in retrieve_similar_faqs(test_q, top_k=3):
    print(f"Score: {r['score']:.3f}")
    print("FAQ Q:", r["prompt"])
    print("FAQ A:", r["response"])
    print("-" * 60)

Score: 0.638
FAQ Q: What exactly do you offer in terms of job guarantee?
FAQ A: If you are worried about job guarantee, don't be. Our support team and mentors are available to help you understand the process step by step.
------------------------------------------------------------
Score: 0.615
FAQ Q: What exactly do you offer in terms of job guarantee?
FAQ A: The idea behind our job guarantee setup is to reduce friction for you. Wherever possible, we give multiple options and clear instructions so that you can focus on learning, not admin work.
------------------------------------------------------------
Score: 0.615
FAQ Q: What exactly do you offer in terms of job guarantee?
FAQ A: The idea behind our job guarantee setup is to reduce friction for you. Wherever possible, we give multiple options and clear instructions so that you can focus on learning, not admin work.
------------------------------------------------------------


In [9]:
def answer_question(question: str, top_k: int = 4, debug: bool = False) -> str:
    """
    Return the best FAQ answer for the user's question.
    Uses local embeddings + cosine similarity, no API calls.
    """
    faqs = retrieve_similar_faqs(question, top_k=top_k)

    if debug:
        print("User Question:", question)
        print("\nTop matches:\n")
        for f in faqs:
            print(f"Score: {f['score']:.3f}")
            print("FAQ Q:", f["prompt"])
            print("FAQ A:", f["response"])
            print("-" * 60)
    best = faqs[0]
    if best["score"] < 0.4:
        return "I'm not sure based on the available FAQs."

    return best["response"]

In [10]:
q = input("Enter your question: ")
print("A: ", answer_question(q, top_k=4, debug=True))

Enter your question:  What are the assignments that we get?


User Question: What are the assignments that we get?

Top matches:

Score: 0.687
FAQ Q: What exactly do you offer in terms of assignment?
FAQ A: Overall, you can think of our assignment design as simple, flexible, and learner-first. Once you enroll, you'll see that everything is documented and easy to follow.
------------------------------------------------------------
Score: 0.677
FAQ Q: What exactly do you offer in terms of assignment?
FAQ A: In short, our assignment system is built to support your learning journey, not get in the way. If you ever feel stuck, you can raise a ticket or ask in the community for quick help.
------------------------------------------------------------
Score: 0.669
FAQ Q: What exactly do you offer in terms of assignment?
FAQ A: We regularly review how our assignment is working in real life and update our processes so that learners like you have a smooth and predictable experience.
------------------------------------------------------------
Score: 0.659
F